In [ ]:
%%capture
!pip install datasets
!pip install simpletransformers

In [ ]:
from simpletransformers.t5 import T5Model

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset('squad')

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [ ]:
raw_datasets['train'][0]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [ ]:
import pandas as pd

In [ ]:
# Converting the training dataset to dataframe
train_list = list()
for i in range(len(raw_datasets['train'])):
  new_record = dict()
  new_record['question'] = raw_datasets['train'][i]['question']
  new_record['context'] = raw_datasets['train'][i]['context']
  new_record['answer'] = raw_datasets['train'][i]['answers']['text'][0]
  train_list.append(new_record)

In [ ]:
# Converting the validation dataset to dataframe
validation_list = list()
for i in range(len(raw_datasets['validation'])):
  new_record = dict()
  new_record['question'] = raw_datasets['validation'][i]['question']
  new_record['context'] = raw_datasets['validation'][i]['context']
  new_record['answer'] = raw_datasets['validation'][i]['answers']['text'][0]
  validation_list.append(new_record)

In [ ]:
train_df = pd.DataFrame.from_dict(train_list)
validation_df = pd.DataFrame.from_dict(validation_list)

In [ ]:
# Adding prefix token to dataframe
train_df['prefix'] = 'generate_question'

In [ ]:
# Adding prefix token to dataframe
validation_df['prefix'] = 'generate_question'

In [ ]:
backup_train_df = train_df.copy()
train_df = train_df.rename(columns={'context': 'input_text', 'question': 'target_text'})  
train_df = train_df.drop('answer', axis=1)


In [ ]:
backup_validation_df = validation_df.copy()
validation_df = validation_df.rename(columns={'context': 'input_text', 'question': 'target_text'})  
validation_df = validation_df.drop('answer', axis=1)


In [ ]:
train_df.head()

,target_text,input_text,prefix
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",generate_question
1,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",generate_question
2,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",generate_question
3,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",generate_question
4,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",generate_question


In [ ]:
validation_df.head()

,target_text,input_text,prefix
0,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,generate_question
1,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,generate_question
2,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,generate_question
3,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,generate_question
4,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,generate_question


In [ ]:
# Model training arguments
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 8,
    "num_train_epochs": 1,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "fp16": False

    "wandb_project": "Squad Question Generation with T5",
}

model = T5Model("t5", "t5-base", args=model_args)

model.train_model(train_df, eval_data=validation_df)

  0%|          | 0/87599 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3282: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Using Adafactor for T5


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running Epoch 0 of 1:   0%|          | 0/10950 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3282: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


(10950,
 {'eval_loss': [2.265967132284133],
  'global_step': [10950],
  'train_loss': [2.153764247894287]})

In [ ]:
model = T5Model("t5", "outputs/best_model", args=model_args)


In [ ]:
to_predict = [
    "generate_question: In 1971, George Lucas wanted to film an adaptation of the Flash Gordon serial, but could not obtain the rights, so he began developing his own space opera.",
]

predictions = model.predict(to_predict)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3282: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions

["What was George Lucas's first space opera?"]

In [ ]:
# Compressing the best model
!tar -zcvf model.tar.gz outputs/best_model/

outputs/best_model/
outputs/best_model/training_args.bin
outputs/best_model/spiece.model
outputs/best_model/optimizer.pt
outputs/best_model/eval_results.txt
outputs/best_model/config.json
outputs/best_model/tokenizer_config.json
outputs/best_model/pytorch_model.bin
outputs/best_model/scheduler.pt
outputs/best_model/special_tokens_map.json
outputs/best_model/model_args.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Saving model to drive
!cp model.tar.gz /content/drive/MyDrive/Models